In [0]:
dbutils.fs.rm("/FileStore/tables/scd2demo", recurse=True)

Out[1]: True

In [0]:
%sql
-- DROP TABLE IF EXISTS scd2demo;
CREATE OR REPLACE TABLE scd2demo (
  pk1 INT,
  pk2 STRING,
  dim1 INT,
  dim2 INT,
  dim3 INT,
  dim4 INT,
  active_status STRING,
  start_date TIMESTAMP,
  end_date TIMESTAMP
)
USING DELTA
LOCATION '/FileStore/tables/scd2demo'

The below operation will create 6 different active Partition Files. Check file path catalog

In [0]:
%sql
INSERT INTO scd2demo VALUES (111, 'Unit1', 200, 500, 800, 400, 'Y', current_timestamp(), '9999-12-31');
INSERT INTO scd2demo VALUES (222, 'Unit2', 900, NULL, 700, 100, 'Y', current_timestamp(), '9999-12-31');
INSERT INTO scd2demo VALUES (333, 'Unit3', 300, 900, 250, 650, 'Y', current_timestamp(), '9999-12-31');
INSERT INTO scd2demo VALUES (666, 'Unit6', 200, 500, 800, 400, 'Y', current_timestamp(), '9999-12-31');
INSERT INTO scd2demo VALUES (777, 'Unit7', 900, NULL, 700, 100, 'Y', current_timestamp(), '9999-12-31');
INSERT INTO scd2demo VALUES (888, 'Unit8', 300, 900, 250, 650, 'Y', current_timestamp(), '9999-12-31');

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
SELECT * FROM scd2demo

pk1,pk2,dim1,dim2,dim3,dim4,active_status,start_date,end_date
111,Unit1,200,500,800,400,Y,2025-11-30T15:04:12.977+0000,9999-12-31T00:00:00.000+0000
333,Unit3,300,900,250,650,Y,2025-11-30T15:04:24.928+0000,9999-12-31T00:00:00.000+0000
666,Unit6,200,500,800,400,Y,2025-11-30T15:04:28.280+0000,9999-12-31T00:00:00.000+0000
888,Unit8,300,900,250,650,Y,2025-11-30T15:04:36.860+0000,9999-12-31T00:00:00.000+0000
222,Unit2,900,null,700,100,Y,2025-11-30T15:04:21.032+0000,9999-12-31T00:00:00.000+0000
777,Unit7,900,null,700,100,Y,2025-11-30T15:04:31.675+0000,9999-12-31T00:00:00.000+0000


The below operation will make one partition file inactive/obselete, so total 5 active files now and One Inactive 

In [0]:
%sql
DELETE FROM scd2demo WHERE pk1=777

num_affected_rows
1


The below operation will make create a new partition file and make the older one inactive/obselete, so total 5 active files and two inactive file (One from DELETE and one from UPDATE)

In [0]:
%sql
UPDATE scd2demo
SET dim1=100 WHERE pk1=666

num_affected_rows
1


The below operation will make create combine all the 5 active file into 1 single Active file. So now we have total 8 Partition files (1 combined active file and 5 previously active files and 2 inactive files)

In [0]:
%sql
OPTIMIZE scd2demo

path,metrics
dbfs:/FileStore/tables/scd2demo,"List(1, 5, List(2705, 2705, 2705.0, 1, 2705), List(2489, 2525, 2517.8, 5, 12589), 0, null, 1, 5, 0, true, 0, 0, 1764515156418, 1764515165507, 8, 1, null, List(0, 0), 9, 9, 1623)"


In [0]:
%sql
DESCRIBE HISTORY scd2demo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
9,2025-11-30T15:06:02.000+0000,6333491925723577,srishtishetty53@gmail.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> false)",null,List(2912376393364894),1130-144856-52x63onm,8,SnapshotIsolation,false,"Map(numRemovedFiles -> 5, numRemovedBytes -> 12589, p25FileSize -> 2705, numDeletionVectorsRemoved -> 0, minFileSize -> 2705, numAddedFiles -> 1, maxFileSize -> 2705, p75FileSize -> 2705, p50FileSize -> 2705, numAddedBytes -> 2705)",null,Databricks-Runtime/12.2.x-scala2.12
8,2025-11-30T15:05:47.000+0000,6333491925723577,srishtishetty53@gmail.com,UPDATE,"Map(predicate -> [""(pk1#4794 = 666)""])",null,List(2912376393364894),1130-144856-52x63onm,7,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 2525, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2433, scanTimeMs -> 1411, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 2525, rewriteTimeMs -> 1020)",null,Databricks-Runtime/12.2.x-scala2.12
7,2025-11-30T15:05:36.000+0000,6333491925723577,srishtishetty53@gmail.com,DELETE,"Map(predicate -> [""(pk1#4177 = 777)""])",null,List(2912376393364894),1130-144856-52x63onm,6,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 2489, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2341, numDeletedRows -> 1, scanTimeMs -> 1490, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 828)",null,Databricks-Runtime/12.2.x-scala2.12
6,2025-11-30T15:04:39.000+0000,6333491925723577,srishtishetty53@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2912376393364894),1130-144856-52x63onm,5,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 2525)",null,Databricks-Runtime/12.2.x-scala2.12
5,2025-11-30T15:04:33.000+0000,6333491925723577,srishtishetty53@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2912376393364894),1130-144856-52x63onm,4,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 2489)",null,Databricks-Runtime/12.2.x-scala2.12
4,2025-11-30T15:04:30.000+0000,6333491925723577,srishtishetty53@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2912376393364894),1130-144856-52x63onm,3,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 2525)",null,Databricks-Runtime/12.2.x-scala2.12
3,2025-11-30T15:04:26.000+0000,6333491925723577,srishtishetty53@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2912376393364894),1130-144856-52x63onm,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 2525)",null,Databricks-Runtime/12.2.x-scala2.12
2,2025-11-30T15:04:23.000+0000,6333491925723577,srishtishetty53@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2912376393364894),1130-144856-52x63onm,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 2489)",null,Databricks-Runtime/12.2.x-scala2.12
1,2025-11-30T15:04:17.000+0000,6333491925723577,srishtishetty53@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2912376393364894),1130-144856-52x63onm,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 2525)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-11-30T15:03:46.000+0000,6333491925723577,srishtishetty53@gmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(2912376393364894),1130-144856-52x63onm,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
%sql
SELECT * FROM scd2demo

pk1,pk2,dim1,dim2,dim3,dim4,active_status,start_date,end_date
111,Unit1,200,500,800,400,Y,2025-11-30T15:04:12.977+0000,9999-12-31T00:00:00.000+0000
222,Unit2,900,null,700,100,Y,2025-11-30T15:04:21.032+0000,9999-12-31T00:00:00.000+0000
333,Unit3,300,900,250,650,Y,2025-11-30T15:04:24.928+0000,9999-12-31T00:00:00.000+0000
666,Unit6,100,500,800,400,Y,2025-11-30T15:04:28.280+0000,9999-12-31T00:00:00.000+0000
888,Unit8,300,900,250,650,Y,2025-11-30T15:04:36.860+0000,9999-12-31T00:00:00.000+0000


### Use `Vacuum` Command to `cleanup` old inactive files

In [0]:
%fs
ls /FileStore/tables/scd2demo

path,name,size,modificationTime
dbfs:/FileStore/tables/scd2demo/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/tables/scd2demo/part-00000-18e64162-238b-4cfd-9174-2de7b59bdd57-c000.snappy.parquet,part-00000-18e64162-238b-4cfd-9174-2de7b59bdd57-c000.snappy.parquet,2489,1764515073000
dbfs:/FileStore/tables/scd2demo/part-00000-48f4c5a3-b7df-4352-a265-7ac6c5936683-c000.snappy.parquet,part-00000-48f4c5a3-b7df-4352-a265-7ac6c5936683-c000.snappy.parquet,2525,1764515078000
dbfs:/FileStore/tables/scd2demo/part-00000-74da37d2-9d76-40e0-a8e4-c477743ec1b5-c000.snappy.parquet,part-00000-74da37d2-9d76-40e0-a8e4-c477743ec1b5-c000.snappy.parquet,2489,1764515062000
dbfs:/FileStore/tables/scd2demo/part-00000-8f9d42ea-5a5c-40ea-ae21-a7873ecd915d-c000.snappy.parquet,part-00000-8f9d42ea-5a5c-40ea-ae21-a7873ecd915d-c000.snappy.parquet,2525,1764515066000
dbfs:/FileStore/tables/scd2demo/part-00000-8fbe5326-179f-459f-be60-88bac1a624de-c000.snappy.parquet,part-00000-8fbe5326-179f-459f-be60-88bac1a624de-c000.snappy.parquet,2705,1764515162000
dbfs:/FileStore/tables/scd2demo/part-00000-c6609d41-bc23-4154-b7fa-c58c22f8b871-c000.snappy.parquet,part-00000-c6609d41-bc23-4154-b7fa-c58c22f8b871-c000.snappy.parquet,2525,1764515069000
dbfs:/FileStore/tables/scd2demo/part-00000-d46e2ace-e440-4ccf-ae58-8354f94b3152-c000.snappy.parquet,part-00000-d46e2ace-e440-4ccf-ae58-8354f94b3152-c000.snappy.parquet,2525,1764515146000
dbfs:/FileStore/tables/scd2demo/part-00000-f80919ef-ae34-4eea-8edd-9f0a4c61d7a3-c000.snappy.parquet,part-00000-f80919ef-ae34-4eea-8edd-9f0a4c61d7a3-c000.snappy.parquet,2525,1764515057000


In [0]:
%sql
VACUUM scd2demo DRY RUN  -- Will show which files will be deleted, it wont delete it

path


- When you run %sql VACUUM scd2demo DRY RUN and it shows "**no rows returned**," it means that there are currently no files eligible for deletion according to the Delta Lake retention policy.
- Default retention is 7 days

In [0]:
%sql
VACUUM scd2demo RETAIN 720 HOURS DRY RUN  -- 720 hours: 30 days retention period

path


In [0]:
%sql
VACUUM scd2demo RETAIN 0 HOURS DRY RUN  -- not recommended with such low retention period, this is just for an example 

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-6140800022841286>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-6140800022841286>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("VkFDVVVNIHNjZDJkZW1vIFJFVEFJTiAwIEhPVVJTIERSWSBSVU4gIC0tIG5vdCByZWNvbW1lbmRlZCB3aXRoIHN1Y2ggbG93IHJldGVudGlvbiBwZXJpb2QsIHRoaXMgaXMganVzdCBmb3IgYW4gZXhhbXBsZQ==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         modu

In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false") # spark

In [0]:
spark.conf.get("spark.databricks.delta.retentionDurationCheck.enabled") # spark

Out[14]: 'false'

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false; -- SQL

key,value
spark.databricks.delta.retentionDurationCheck.enabled,false


In [0]:
%sql
VACUUM scd2demo RETAIN 0 HOURS DRY RUN  -- now this wont throw and error bcz we set the config above, it will show 7 inactive files to be cleaned up

path
dbfs:/FileStore/tables/scd2demo/part-00000-d46e2ace-e440-4ccf-ae58-8354f94b3152-c000.snappy.parquet
dbfs:/FileStore/tables/scd2demo/part-00000-c6609d41-bc23-4154-b7fa-c58c22f8b871-c000.snappy.parquet
dbfs:/FileStore/tables/scd2demo/part-00000-18e64162-238b-4cfd-9174-2de7b59bdd57-c000.snappy.parquet
dbfs:/FileStore/tables/scd2demo/part-00000-8f9d42ea-5a5c-40ea-ae21-a7873ecd915d-c000.snappy.parquet
dbfs:/FileStore/tables/scd2demo/part-00000-f80919ef-ae34-4eea-8edd-9f0a4c61d7a3-c000.snappy.parquet
dbfs:/FileStore/tables/scd2demo/part-00000-48f4c5a3-b7df-4352-a265-7ac6c5936683-c000.snappy.parquet
dbfs:/FileStore/tables/scd2demo/part-00000-74da37d2-9d76-40e0-a8e4-c477743ec1b5-c000.snappy.parquet


> Note: 

Lowering the retention period below 7 days can lead to data loss if there are still readers using the old files. Use the VACUUM command with a lower retention period only if you are certain no one needs the old data

In [0]:
%sql
MSCK REPAIR TABLE scd2demo

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6140800022841292>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-6140800022841292>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("TVNDSyBSRVBBSVIgVEFCTEUgc2NkMmRlbW8=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /datab

In [0]:
%sql
VACUUM scd2demo 

path
dbfs:/FileStore/tables/scd2demo


In [0]:
%sql
REFRESH TABLE scd2demo

After VACUUM, it shd have removed the 7 inactive files not sure why it did not, check on this

In [0]:
%fs
ls /FileStore/tables/scd2demo

path,name,size,modificationTime
dbfs:/FileStore/tables/scd2demo/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/tables/scd2demo/part-00000-18e64162-238b-4cfd-9174-2de7b59bdd57-c000.snappy.parquet,part-00000-18e64162-238b-4cfd-9174-2de7b59bdd57-c000.snappy.parquet,2489,1764515073000
dbfs:/FileStore/tables/scd2demo/part-00000-48f4c5a3-b7df-4352-a265-7ac6c5936683-c000.snappy.parquet,part-00000-48f4c5a3-b7df-4352-a265-7ac6c5936683-c000.snappy.parquet,2525,1764515078000
dbfs:/FileStore/tables/scd2demo/part-00000-74da37d2-9d76-40e0-a8e4-c477743ec1b5-c000.snappy.parquet,part-00000-74da37d2-9d76-40e0-a8e4-c477743ec1b5-c000.snappy.parquet,2489,1764515062000
dbfs:/FileStore/tables/scd2demo/part-00000-8f9d42ea-5a5c-40ea-ae21-a7873ecd915d-c000.snappy.parquet,part-00000-8f9d42ea-5a5c-40ea-ae21-a7873ecd915d-c000.snappy.parquet,2525,1764515066000
dbfs:/FileStore/tables/scd2demo/part-00000-8fbe5326-179f-459f-be60-88bac1a624de-c000.snappy.parquet,part-00000-8fbe5326-179f-459f-be60-88bac1a624de-c000.snappy.parquet,2705,1764515162000
dbfs:/FileStore/tables/scd2demo/part-00000-c6609d41-bc23-4154-b7fa-c58c22f8b871-c000.snappy.parquet,part-00000-c6609d41-bc23-4154-b7fa-c58c22f8b871-c000.snappy.parquet,2525,1764515069000
dbfs:/FileStore/tables/scd2demo/part-00000-d46e2ace-e440-4ccf-ae58-8354f94b3152-c000.snappy.parquet,part-00000-d46e2ace-e440-4ccf-ae58-8354f94b3152-c000.snappy.parquet,2525,1764515146000
dbfs:/FileStore/tables/scd2demo/part-00000-f80919ef-ae34-4eea-8edd-9f0a4c61d7a3-c000.snappy.parquet,part-00000-f80919ef-ae34-4eea-8edd-9f0a4c61d7a3-c000.snappy.parquet,2525,1764515057000
